In [176]:
# open mysql connection

In [177]:
import pymysql
import pandas as pd
import numpy as np

In [178]:
conn = pymysql.connect(host='localhost', user='root', password='root', db='partsio-schema', charset='utf8mb4')
cat_id=3

components = pd.read_sql_query("select * from components where category=%(cat_id)s;", conn, params={'cat_id': cat_id})
cat_def = pd.read_sql_query("select * from cat_def where category=%(cat_id)s;", conn, params={'cat_id': cat_id})
test_type = pd.read_sql_query("select * from test_type where cat_id=%(cat_id)s;", conn, params={'cat_id': cat_id})
score_ref = pd.read_sql_query("select * from score_ref order by difference;", conn)
col_name_ref = pd.read_sql_query("select * from col_ref", conn)

In [179]:
conn.close()

In [180]:
# if component count <=1, stop
#if components.shape[0] == 0:
    # return with no data
#elif components.shape[0] == 1:
    # format one row and display, return    

In [181]:
#pf_test_cols = ['mpn','package_size','polarity','v_esd_air','v_esd_contact','v_br_v_trig_min','i_holding_min','c_j_max','c_j_delta_max','l_io_max','i_leakage_max']
pf_test_cols = test_type.loc[test_type['test_type'] == 'P/F','param_name'].tolist()
pf_test_cols.insert(0,'mpn')

pf_test = pd.DataFrame(columns=pf_test_cols)
pf_test['mpn'] = components.loc[:,'mpn'].copy()
pf_test

,mpn,package_size,polarity,v_esd_air,v_esd_contact,v_br_v_trig_min,i_holding_min,c_j_max,c_j_delta_max,i_leakage_max
0,ESD106-B1-W0201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ESD108-B1-CSP0201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ESD131-B1-W0201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ESD134-B1-W0201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PESD3V3Z1BSF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PESD7V0C1BSF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,PESD5V0C1BSF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ESD8011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,RClamp3371ZC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,RClamp5031ZA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [182]:
if 'package_size' in pf_test:
    pf_test['package_size'] = components['package_size'] == cat_def.loc[0,'package_size']
if 'polarity' in pf_test:
    pf_test['polarity'] = components['polarity'] == cat_def.loc[0,'polarity']
if 'v_esd_air' in pf_test:
    pf_test['v_esd_air'] = components['v_esd_air'] >= cat_def.loc[0,'v_esd_air']
if 'v_esd_contact' in pf_test:
    pf_test['v_esd_contact'] = components['v_esd_contact'] >= cat_def.loc[0,'v_esd_contact']
if 'v_br_v_trig_min' in pf_test:
    pf_test['v_br_v_trig_min'] = components['v_br_v_trig_min'] >= cat_def.loc[0,'v_br_v_trig_min']
if 'i_holding_min' in pf_test:
    i_holding_test = (components['design_topology'] == 'SCR') | (components['design_topology'] == 'Deep Snapback') | ((pd.notnull(components['v_holding_min'])) & (components['v_holding_min'] < components['v_rwm_max'])) | (components[['tlp_clamp_voltage_4a_reverse','tlp_clamp_voltage_8a_reverse','tlp_clamp_voltage_16a_reverse','tlp_clamp_voltage_30a_reverse']].min(axis=1) < components['v_rwm_max'])
    i_holding_final = components['i_holding_min'] >= cat_def.loc[0,'i_holding_min']
    pf_test.loc[i_holding_test == True, 'i_holding_min'] = i_holding_final
    pf_test.loc[i_holding_test == False, 'i_holding_min'] = 'N/A'
if 'c_j_max' in pf_test:
    pf_test['c_j_max'] = components['c_j_max'] <= cat_def.loc[0,'c_j_max']
if 'c_j_delta_max' in pf_test:
    cj_delta_notnull = pd.notnull(components['c_j_delta_max'])
    cj_delta_test = components['c_j_delta_max'] <= int(cat_def.loc[0,'c_j_delta_max'])
    pf_test.loc[cj_delta_notnull == True, 'c_j_delta_max'] = cj_delta_test
    pf_test.loc[cj_delta_notnull == False, 'c_j_delta_max'] = 'No Data'
if 'l_io_max' in pf_test:
    l_io_max_notnull = pd.notnull(components['l_io_max'])
    #l_io_max_test = components['l_io_max'] <= cat_def.loc[0,'l_io_max']
    pf_test.loc[l_io_max_notnull == True, 'l_io_max'] = components['l_io_max'] <= cat_def.loc[0,'l_io_max']
    pf_test.loc[l_io_max_notnull == False, 'l_io_max'] = 'No Data'    
if 'i_leakage_max' in pf_test:
    pf_test['i_leakage_max'] = components['i_leakage_max'] <= cat_def.loc[0,'i_leakage_max']
#pf_test['status'] = pf_test[['package_size','polarity','v_esd_air','v_esd_contact','v_br_v_trig_min','i_holding_min','c_j_max','i_leakage_max']].apply(lambda x: 'Fail' if sum(x == False) > 0 else 'Pass', axis=1)
pf_test['status'] = pf_test[list(pf_test.columns)].apply(lambda x: 'Fail' if sum(x == False) > 0 else 'Pass', axis=1)
pf_test

,mpn,package_size,polarity,v_esd_air,v_esd_contact,v_br_v_trig_min,i_holding_min,c_j_max,c_j_delta_max,i_leakage_max,status
0,ESD106-B1-W0201,True,True,False,True,True,N/A,True,No Data,True,Fail
1,ESD108-B1-CSP0201,True,True,True,True,True,N/A,True,No Data,True,Pass
2,ESD131-B1-W0201,True,True,True,True,True,True,True,No Data,True,Pass
3,ESD134-B1-W0201,True,True,True,True,True,True,True,No Data,True,Pass
4,PESD3V3Z1BSF,True,True,True,True,True,True,True,No Data,True,Pass
5,PESD7V0C1BSF,True,True,True,True,True,True,True,No Data,True,Pass
6,PESD5V0C1BSF,True,True,True,True,True,True,True,No Data,True,Pass
7,ESD8011,True,True,True,True,True,True,True,No Data,False,Fail
8,RClamp3371ZC,True,True,False,False,True,True,True,True,True,Fail
9,RClamp5031ZA,True,True,True,True,True,True,True,True,True,Pass


In [183]:
#score_test_cols = ['mpn','v_br_v_trig_max','r_dyn_forward','r_dyn_reverse','turn_on_time_max','peak_8kv_contact_forward','peak_8kv_contact_reverse','tlp_clamp_voltage_4a_forward','tlp_clamp_voltage_8a_forward','tlp_clamp_voltage_16a_forward','tlp_clamp_voltage_30a_forward','tlp_clamp_voltage_4a_reverse','tlp_clamp_voltage_8a_reverse','tlp_clamp_voltage_16a_reverse','tlp_clamp_voltage_30a_reverse','insertion_loss_1ghz','insertion_loss_25ghz','insertion_loss_5ghz','mhz_824_890_2nd_harmonics_20dbm_level','mhz_824_890_3rd_harmonics_20dbm_level','ghz_24_2484_2nd_harmonics_20dbm_level','ghz_24_2484_3rd_harmonics_20dbm_level','ghz_5_5825_2nd_harmonics_20dbm_level','ghz_5_5825_3rd_harmonics_20dbm_level']

score_test_cols = test_type.loc[test_type['test_type'] == 'Score','param_name'].tolist()
score_test_cols.insert(0,'mpn')
score_test_cols.append('status')
score_test_cols.append('score')

score_test = pd.DataFrame(columns=score_test_cols)
score_test['mpn'] = components.loc[:,'mpn'].copy()
score_test['status'] = pf_test.loc[:,'status'].copy()
score_test

,mpn,v_br_v_trig_max,r_dyn_forward,r_dyn_reverse,turn_on_time_max,peak_8kv_contact_forward,peak_8kv_contact_reverse,tlp_clamp_voltage_4a_forward,tlp_clamp_voltage_8a_forward,tlp_clamp_voltage_16a_forward,...,insertion_loss_25ghz,insertion_loss_5ghz,mhz_824_890_2nd_harmonics_20dbm_level,mhz_824_890_3rd_harmonics_20dbm_level,ghz_24_2484_2nd_harmonics_20dbm_level,ghz_24_2484_3rd_harmonics_20dbm_level,ghz_5_5825_2nd_harmonics_20dbm_level,ghz_5_5825_3rd_harmonics_20dbm_level,status,score
0,ESD106-B1-W0201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,NaN
1,ESD108-B1-CSP0201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,NaN
2,ESD131-B1-W0201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,NaN
3,ESD134-B1-W0201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,NaN
4,PESD3V3Z1BSF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,NaN
5,PESD7V0C1BSF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,NaN
6,PESD5V0C1BSF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,NaN
7,ESD8011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,NaN
8,RClamp3371ZC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,NaN
9,RClamp5031ZA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,NaN


In [184]:
min = 10000
max = -10000
vbr_max = components['v_br_v_trig_max'].copy()
vbr_min = components['v_br_v_trig_min'].copy()
for i, v in vbr_max.items():
    if score_test.loc[i,'status'] == 'Fail':
        score_test.loc[i,'v_br_v_trig_max'] = np.NaN
    elif pd.isnull(v) & pd.isnull(vbr_min[i]):
        score_test.loc[i,'status'] = 'Fail'
        score_test.loc[i,'v_br_v_trig_max'] = np.NaN
    elif pd.notnull(v):
        score_test.loc[i,'v_br_v_trig_max'] = vbr_max[i]
    else:
        score_test.loc[i,'v_br_v_trig_max'] = vbr_min[i] * 2
score_test

,mpn,v_br_v_trig_max,r_dyn_forward,r_dyn_reverse,turn_on_time_max,peak_8kv_contact_forward,peak_8kv_contact_reverse,tlp_clamp_voltage_4a_forward,tlp_clamp_voltage_8a_forward,tlp_clamp_voltage_16a_forward,...,insertion_loss_25ghz,insertion_loss_5ghz,mhz_824_890_2nd_harmonics_20dbm_level,mhz_824_890_3rd_harmonics_20dbm_level,ghz_24_2484_2nd_harmonics_20dbm_level,ghz_24_2484_3rd_harmonics_20dbm_level,ghz_5_5825_2nd_harmonics_20dbm_level,ghz_5_5825_3rd_harmonics_20dbm_level,status,score
0,ESD106-B1-W0201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,NaN
1,ESD108-B1-CSP0201,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,NaN
2,ESD131-B1-W0201,10.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,NaN
3,ESD134-B1-W0201,11.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,NaN
4,PESD3V3Z1BSF,9.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,NaN
5,PESD7V0C1BSF,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,NaN
6,PESD5V0C1BSF,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,NaN
7,ESD8011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,NaN
8,RClamp3371ZC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,NaN
9,RClamp5031ZA,10.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pass,NaN


In [185]:
max_cols = ['mhz_824_890_2nd_harmonics_20dbm_level','mhz_824_890_3rd_harmonics_20dbm_level','ghz_24_2484_2nd_harmonics_20dbm_level','ghz_24_2484_3rd_harmonics_20dbm_level','ghz_5_5825_2nd_harmonics_20dbm_level','ghz_5_5825_3rd_harmonics_20dbm_level']
for column in score_test:
    if (column == 'mpn') | (column == 'status') | (column == 'score'):
        continue
    else:
        col_status = score_test['status'].copy()
        col_null = pd.notnull(components[column])
        col_component = components[column].copy()
        if column != 'v_br_v_trig_max':           
            col_component.loc[col_status == 'Fail'] = np.NaN
            score_test.loc[col_null == True, column] = col_component
            score_test.loc[col_null == False, column] = np.NaN
            score_test[column] = col_component.copy()
        min = score_test[column].min()
        max = score_test[column].max()
        max_score = test_type.loc[test_type['param_name'] == column,'max_score'].values[0]
        #col_ref = 'ratio' + str(test_type.loc[test_type['param_name'] == column,'score_ref_col'])
        col_ref_num = test_type.loc[test_type['param_name'] == column,'score_ref_col'].values[0]
        col_ref = score_ref['ratio'+str(col_ref_num)].copy()
        diff = score_ref['difference'].tolist()
        if column in max_cols:
            calc_col = ((score_test[column] - max) * -1) / max
        else:
            calc_col = (score_test[column] - min) / min
        #calc_col.fillna(-1,inplace=True)
        #calc_lookup = pd.merge_asof(calc_col,score_ref,left_on=column, right_on='difference', direction='forward')
        for i, v in calc_col.items():
            if pd.isnull(v):
                continue
            else:
                score_row = pd.Index(diff).get_loc(v, method='pad')
                #score_row = score_ref['difference'].Index.get_loc(v, method='bfill')
                #print(column, v, max_score, col_ref_num, score_row, col_ref[score_row], score_test[column])
                score_test.loc[i,column] =  max_score * col_ref[score_row]
score_test['score'] = score_test.apply(lambda x: pd.to_numeric(x, errors='coerce')).sum(axis=1)
#score_test['score'] = score_test.sum(numeric_only=True, axis=1)

In [186]:
score_test

,mpn,v_br_v_trig_max,r_dyn_forward,r_dyn_reverse,turn_on_time_max,peak_8kv_contact_forward,peak_8kv_contact_reverse,tlp_clamp_voltage_4a_forward,tlp_clamp_voltage_8a_forward,tlp_clamp_voltage_16a_forward,...,insertion_loss_25ghz,insertion_loss_5ghz,mhz_824_890_2nd_harmonics_20dbm_level,mhz_824_890_3rd_harmonics_20dbm_level,ghz_24_2484_2nd_harmonics_20dbm_level,ghz_24_2484_3rd_harmonics_20dbm_level,ghz_5_5825_2nd_harmonics_20dbm_level,ghz_5_5825_3rd_harmonics_20dbm_level,status,score
0,ESD106-B1-W0201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,0.00
1,ESD108-B1-CSP0201,2.25,0.00,0.00,NaN,0.0,0.0,0.0,0.0,0.0,...,1.80,1.95,0.0,0.0,0.0,0.0,NaN,NaN,Pass,7.65
2,ESD131-B1-W0201,3.5,0.00,0.00,NaN,0.0,0.0,3.0,0.0,0.0,...,2.10,1.65,0.0,0.0,0.0,0.0,NaN,NaN,Pass,18.25
3,ESD134-B1-W0201,3.25,2.00,2.00,NaN,0.0,0.0,7.5,6.0,3.5,...,2.55,1.65,NaN,NaN,NaN,NaN,NaN,NaN,Pass,56.45
4,PESD3V3Z1BSF,4.25,4.50,4.50,NaN,10.0,10.0,10.0,10.0,5.0,...,0.00,0.00,0.3,1.8,0.6,0.6,NaN,NaN,Pass,95.20
5,PESD7V0C1BSF,2,3.25,3.25,NaN,5.5,6.0,10.0,9.5,5.0,...,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,Pass,75.50
6,PESD5V0C1BSF,2,3.25,3.25,NaN,5.5,6.0,10.0,9.5,5.0,...,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,Pass,75.50
7,ESD8011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,0.00
8,RClamp3371ZC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,0.00
9,RClamp5031ZA,3.5,5.00,5.00,NaN,9.5,10.0,6.0,6.0,4.0,...,0.00,0.00,3.0,3.0,3.0,3.0,NaN,NaN,Pass,85.95


In [187]:
score_test['rank'] = score_test.score[score_test['score']>0].rank(ascending=False, method='dense')
score_test

,mpn,v_br_v_trig_max,r_dyn_forward,r_dyn_reverse,turn_on_time_max,peak_8kv_contact_forward,peak_8kv_contact_reverse,tlp_clamp_voltage_4a_forward,tlp_clamp_voltage_8a_forward,tlp_clamp_voltage_16a_forward,...,insertion_loss_5ghz,mhz_824_890_2nd_harmonics_20dbm_level,mhz_824_890_3rd_harmonics_20dbm_level,ghz_24_2484_2nd_harmonics_20dbm_level,ghz_24_2484_3rd_harmonics_20dbm_level,ghz_5_5825_2nd_harmonics_20dbm_level,ghz_5_5825_3rd_harmonics_20dbm_level,status,score,rank
0,ESD106-B1-W0201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,0.00,NaN
1,ESD108-B1-CSP0201,2.25,0.00,0.00,NaN,0.0,0.0,0.0,0.0,0.0,...,1.95,0.0,0.0,0.0,0.0,NaN,NaN,Pass,7.65,9.0
2,ESD131-B1-W0201,3.5,0.00,0.00,NaN,0.0,0.0,3.0,0.0,0.0,...,1.65,0.0,0.0,0.0,0.0,NaN,NaN,Pass,18.25,7.0
3,ESD134-B1-W0201,3.25,2.00,2.00,NaN,0.0,0.0,7.5,6.0,3.5,...,1.65,NaN,NaN,NaN,NaN,NaN,NaN,Pass,56.45,4.0
4,PESD3V3Z1BSF,4.25,4.50,4.50,NaN,10.0,10.0,10.0,10.0,5.0,...,0.00,0.3,1.8,0.6,0.6,NaN,NaN,Pass,95.20,1.0
5,PESD7V0C1BSF,2,3.25,3.25,NaN,5.5,6.0,10.0,9.5,5.0,...,0.00,NaN,NaN,NaN,NaN,NaN,NaN,Pass,75.50,3.0
6,PESD5V0C1BSF,2,3.25,3.25,NaN,5.5,6.0,10.0,9.5,5.0,...,0.00,NaN,NaN,NaN,NaN,NaN,NaN,Pass,75.50,3.0
7,ESD8011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,0.00,NaN
8,RClamp3371ZC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fail,0.00,NaN
9,RClamp5031ZA,3.5,5.00,5.00,NaN,9.5,10.0,6.0,6.0,4.0,...,0.00,3.0,3.0,3.0,3.0,NaN,NaN,Pass,85.95,2.0


In [189]:
components['status'] = score_test['status'].copy()
components['score'] = score_test['score'].copy()
components['rank'] = score_test['rank'].copy()

sorted_components = components.sort_values('rank')
sorted_components = sorted_components.dropna(axis=1, how='all')
sorted_components = sorted_components.rename(columns=dict(zip(col_name_ref['col_name'], col_name_ref['final_name'])))

#sorted_components
output = sorted_components[['Category','Manufacturer','MPN','Status','Rank']].copy()
output

# jsonify for frontend


,Category,Manufacturer,MPN,Status,Rank
4,3,Nexperia,PESD3V3Z1BSF,Pass,1.0
9,3,Semtech,RClamp5031ZA,Pass,2.0
5,3,Nexperia,PESD7V0C1BSF,Pass,3.0
6,3,Nexperia,PESD5V0C1BSF,Pass,3.0
3,3,Infineon,ESD134-B1-W0201,Pass,4.0
13,3,Toshiba,DF2B6M5SL,Pass,5.0
10,3,Toshiba,DF2B6M4SL,Pass,6.0
2,3,Infineon,ESD131-B1-W0201,Pass,7.0
11,3,Toshiba,DF2B6M4ASL,Pass,8.0
1,3,Infineon,ESD108-B1-CSP0201,Pass,9.0
